In [3]:
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csc_matrix
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import umap
import random
import multiprocessing
random.seed(123)
from joblib import Parallel, delayed
import warnings
#warnings.filterwarnings('ignore')
import os


In [2]:
def read_mat(h5_con):
    mat = scs.csc_matrix(
        (h5_con['matrix']['data'][:], # Count values
         h5_con['matrix']['indices'][:], # Row indices
         h5_con['matrix']['indptr'][:]), # Pointers for column positions
        shape = tuple(h5_con['matrix']['shape'][:]) # Matrix dimensions
    )
    return mat


def read_obs(h5con):
    bc = h5con['matrix']['barcodes'][:]
    bc = [x.decode('UTF-8') for x in bc]

    # Initialized the DataFrame with cell barcodes
    obs_df = pd.DataFrame({ 'barcodes' : bc })

    # Get the list of available metadata columns
    obs_columns = h5con['matrix']['observations'].keys()

    # For each column
    for col in obs_columns:
        # Read the values
        values = h5con['matrix']['observations'][col][:]
        # Check for byte storage
        if(isinstance(values[0], (bytes, bytearray))):
            # Decode byte strings
            values = [x.decode('UTF-8') for x in values]
        # Add column to the DataFrame
        obs_df[col] = values
    
    return obs_df
# define a function to construct anndata object from a h5 file
def read_h5_anndata(h5_file):
    h5_con = h5py.File(h5_file, mode = 'r')
    # extract the expression matrix
    mat = read_mat(h5_con)
    # extract gene names
    genes = h5_con['matrix']['features']['name'][:]
    genes = [x.decode('UTF-8') for x in genes]
    # extract metadata
    obs_df = read_obs(h5_con)
    # construct anndata
    adata = anndata.AnnData(mat.T,
                             obs = obs_df)
    # make sure the gene names aligned
    adata.var_names = genes

    adata.var_names_make_unique()
    return adata
def get_last_pattern(inputstr):
    pattern = r"[^/]+(?=$)"
    match = re.search(pattern, inputstr)
    if match:
        return match.group(0)
    else:
        return ""
def process_file(file_name):
    adata = read_h5_anndata(file_name)
    output_file = 'h5ad_flu_after_qc/'+adata.obs['pbmc_sample_id'][0]+'.h5ad'
    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        print(f"File {output_file} already exists and is not empty. Skipping processing.")
        return
    adata.obs=adata.obs.merge(meta_data[col_list], on='pbmc_sample_id', how='left')
    doublet_scores=pd.read_csv('/home/jupyter/BRI_Analysis/scRNA/Doublet_Scores/'+adata.obs['pbmc_sample_id'][0]+'.csv', index_col=0)
    L1_labels=pd.read_csv('/home/jupyter/BRI_Analysis/scRNA/Labels/'+adata.obs['pbmc_sample_id'][0]+'_L1_predicted_labels.csv', index_col=0)
    L2_labels=pd.read_csv('/home/jupyter/BRI_Analysis/scRNA/Labels/'+adata.obs['pbmc_sample_id'][0]+'_L2_predicted_labels.csv', index_col=0)
    L3_labels=pd.read_csv('/home/jupyter/BRI_Analysis/scRNA/Labels/'+adata.obs['pbmc_sample_id'][0]+'_L3_predicted_labels.csv', index_col=0)
    L3_5_labels=pd.read_csv('/home/jupyter/BRI_Analysis/scRNA/Labels/'+adata.obs['pbmc_sample_id'][0]+'_L3.5_predicted_labels.csv', index_col=0)
    L1_labels.columns=[ 'barcodes', 'AIFI_L1']
    L2_labels.columns=[ 'barcodes', 'AIFI_L2']
    L3_labels.columns=[ 'barcodes', 'AIFI_L3']
    L3_5_labels.columns=[ 'barcodes', 'AIFI_L3.5']
    print(doublet_scores['barcodes'].tolist()==adata.obs['barcodes'].tolist(),
          L1_labels['barcodes'].tolist()==adata.obs['barcodes'].tolist(),
          L2_labels['barcodes'].tolist()==adata.obs['barcodes'].tolist(),
          L3_labels['barcodes'].tolist()==adata.obs['barcodes'].tolist(),
          L3_5_labels['barcodes'].tolist()==adata.obs['barcodes'].tolist())
    adata.obs= pd.merge(adata.obs, doublet_scores, on='barcodes', how='left')
    adata.obs= pd.merge(adata.obs, L1_labels, on='barcodes', how='left')
    adata.obs= pd.merge(adata.obs, L2_labels, on='barcodes', how='left')
    adata.obs= pd.merge(adata.obs, L3_labels, on='barcodes', how='left')
    adata.obs= pd.merge(adata.obs, L3_5_labels, on='barcodes', how='left')
    adata.obs.index=adata.obs['barcodes']
    adata.var["mito"] = adata.var_names.str.startswith("MT-")
    adata=adata[adata.obs['predicted_doublet']==False]
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mito"], inplace=True)
    adata=adata[(adata.obs["pct_counts_mito"] <10) & (adata.obs["n_genes"] <5000) &(adata.obs["n_genes"] >200) ]
    adata.write_h5ad('h5ad_flu_after_qc/'+adata.obs['pbmc_sample_id'][0]+'.h5ad')

In [3]:
col_list=['subject.biologicalSex','subject.ethnicity', 'subject.partnerCode',
          'subject.race', 'subject.subjectGuid', 'cohort.cohortGuid', 'sample.visitName', 
          'sample.visitDetails', 'subject.birthYear','CMV.IgG.Serology.Result.Interpretation', 
          'BMI','pbmc_sample_id']

In [4]:
meta_data=pd.read_csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')

# Read all h5 and split into individual H5

In [5]:
def load_file(file_name):
    try:
        result = sc.read_h5ad("/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/" + file_name + '.h5ad')
        return result
    except Exception as e:
        print(f'Error reading {file_name}: {e}')
        return None

In [6]:
%%time
file_names= meta_data["pbmc_sample_id"].tolist()
h5_list = []
with ThreadPoolExecutor(max_workers=60) as executor:
    future_to_file = {executor.submit(load_file, file_name): file_name for file_name in file_names}
    for future in tqdm(as_completed(future_to_file), total=len(file_names)):
        result = future.result()
        if result is not None:
            h5_list.append(result)

100% 868/868 [32:34<00:00,  2.25s/it]  

CPU times: user 2min 40s, sys: 3min 28s, total: 6min 9s
Wall time: 32min 35s


In [7]:
def subset_adata(adata, celltype):
    adata_subset = adata[adata.obs['AIFI_L3'] == celltype].copy()
    return adata_subset

In [34]:
if h5_list[9].obs['AIFI_L3'].nunique() == 71:
    for i in range(0,71):
        celltypes_to_process = [h5_list[9].obs['AIFI_L3'].unique()[i]]
        adata_list = []
        print(celltypes_to_process[0])

        with ThreadPoolExecutor(max_workers=60) as executor:
            future_to_adata = {executor.submit(subset_adata, adata_file, celltype): adata_file for celltype in celltypes_to_process for adata_file in h5_list}
            for future in tqdm(as_completed(future_to_adata), total=len(future_to_adata)):
                result = future.result()
                if result is not None:
                    adata_list.append(result)
        combined= anndata.concat(adata_list)
        combined.write_h5ad('/home/jupyter/BRI_Analysis/h5_by_celltype/'+celltypes_to_process[0]+'.h5ad')

cDC1


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(d

FileNotFoundError: [Errno 2] Unable to create file (unable to open file: name = '/home/jupyter/BRI_Analysis/h5_by_celltype/cDC1.h5ad', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [20]:
h5_list[22].obs['AIFI_L3'].nunique()

70

In [30]:
h5_list[25].obs['AIFI_L3'].nunique() == 71

False

In [24]:
len(h5_list)

868

In [31]:
for i, item in enumerate(h5_list):
    if item.obs['AIFI_L3'].nunique() == 71:
        print(f"Index {i} satisfies the condition.")


Index 9 satisfies the condition.
Index 21 satisfies the condition.
Index 32 satisfies the condition.
Index 60 satisfies the condition.
Index 72 satisfies the condition.
Index 79 satisfies the condition.
Index 122 satisfies the condition.
Index 132 satisfies the condition.
Index 159 satisfies the condition.
Index 175 satisfies the condition.
Index 191 satisfies the condition.
Index 199 satisfies the condition.
Index 259 satisfies the condition.
Index 267 satisfies the condition.
Index 277 satisfies the condition.
Index 297 satisfies the condition.
Index 318 satisfies the condition.
Index 322 satisfies the condition.
Index 362 satisfies the condition.
Index 370 satisfies the condition.
Index 379 satisfies the condition.
Index 380 satisfies the condition.
Index 404 satisfies the condition.
Index 405 satisfies the condition.
Index 439 satisfies the condition.
Index 447 satisfies the condition.
Index 458 satisfies the condition.
Index 460 satisfies the condition.
Index 464 satisfies the con

# Other method

In [35]:
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures
import seaborn as sns
from scipy.sparse import csc_matrix
import random
random.seed(123)
from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed


In [37]:
meta_data=pd.read_csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')
meta_data=meta_data[meta_data['Covid_exlcusion']=='no']
TimePoints=['Flu Year 1 Day 0','Flu Year 1 Day 7','Flu Year 2 Day 0','Flu Year 2 Day 7']
meta_data_subset=meta_data[meta_data['sample.visitName'].isin(TimePoints)]

In [38]:
from tqdm import tqdm
h5_list = []
for i in tqdm(range(len(meta_data_subset["pbmc_sample_id"].tolist()))):
    try:
        result = sc.read_h5ad("/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/"+meta_data_subset["pbmc_sample_id"].tolist()[i]+'.h5ad')
        h5_list.append(result)
        print()
    except Exception as e:
        print(f'Error reading {file_name}: {e}')

  0% 1/352 [00:00<01:13,  4.79it/s]

  1% 2/352 [00:00<01:12,  4.86it/s]

  1% 3/352 [00:00<01:21,  4.29it/s]

  1% 4/352 [00:00<01:28,  3.94it/s]

  1% 5/352 [00:01<01:36,  3.60it/s]

...

 99% 348/352 [01:43<00:01,  3.57it/s]

 99% 349/352 [01:43<00:00,  3.68it/s]

 99% 350/352 [01:43<00:00,  3.58it/s]

100% 351/352 [01:43<00:00,  3.88it/s]

100% 352/352 [01:44<00:00,  3.38it/s]

In [23]:
len(h5_list)

NameError: name 'h5_list' is not defined

In [44]:
%%time
combined= anndata.concat(h5_list)


/opt/conda/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):


CPU times: user 11min 2s, sys: 1min 46s, total: 12min 48s
Wall time: 12min 48s


In [45]:
combined.write_h5ad('flu_raw.h5ad')

In [4]:
combined=sc.read_h5ad('/home/jupyter/Myeloid_cells/files/flu_raw.h5ad')

In [11]:
b_cells_subset = combined.obs[combined.obs['AIFI_L1'] == 'B cell']
b_cells_subset

,barcodes,batch_id,cell_name,cell_uuid,chip_id,hto_barcode,hto_category,n_genes,n_mito_umis,n_reads,...,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mito,log1p_total_counts_mito,pct_counts_mito
barcodes,,,,,,,,,,,,,,,,,,,,,
cfa6417648b611ea8957bafe6d70929e,cfa6417648b611ea8957bafe6d70929e,B001,gasolinic_vocational_adouri,cfa6417648b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,1650,280,15119,...,7.409136,4635.0,8.441607,35.102481,49.751888,60.474649,74.822006,280.0,5.638355,6.040993
cfad979648b611ea8957bafe6d70929e,cfad979648b611ea8957bafe6d70929e,B001,subdermic_hapless_leopard,cfad979648b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,1582,171,14574,...,7.367077,4295.0,8.365439,36.437718,49.592549,60.000000,74.807916,171.0,5.147494,3.981374
cfc2364248b611ea8957bafe6d70929e,cfc2364248b611ea8957bafe6d70929e,B001,flavorous_sterling_bactrian,cfc2364248b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,1728,310,20620,...,7.455298,6155.0,8.725183,39.561332,56.669374,66.547522,78.375305,310.0,5.739793,5.036556
70a8e30448b611eaa65d9e7d578d66f2,70a8e30448b611eaa65d9e7d578d66f2,B001,crumby_mini_genet,70a8e30448b611eaa65d9e7d578d66f2,B001-P1C1,TGATGGCCTATTGGG,singlet,2583,449,37365,...,7.857094,10503.0,9.259511,38.636580,53.956013,63.362849,74.635818,449.0,6.109248,4.274969
70aff80648b611eaa65d9e7d578d66f2,70aff80648b611eaa65d9e7d578d66f2,B001,clownish_like_kiskadee,70aff80648b611eaa65d9e7d578d66f2,B001-P1C1,TGATGGCCTATTGGG,singlet,2492,338,31035,...,7.821242,8950.0,9.099521,37.977654,53.173184,62.458101,73.407821,338.0,5.826000,3.776536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9f5d30e09a2711ed8e94f29eefb82f83,9f5d30e09a2711ed8e94f29eefb82f83,B148,chosen_uncleansed_monarch,9f5d30e09a2711ed8e94f29eefb82f83,B148-P2C3,TGTCTTTCCTGCCAG,singlet,1329,108,13718,...,7.192934,4622.0,8.438799,47.490264,62.115967,71.332756,82.064042,108.0,4.691348,2.336651
9f6f44b09a2711ed8e94f29eefb82f83,9f6f44b09a2711ed8e94f29eefb82f83,B148,wool_simple_lobo,9f6f44b09a2711ed8e94f29eefb82f83,B148-P2C3,TGTCTTTCCTGCCAG,singlet,1311,103,11260,...,7.179308,3735.0,8.225771,40.240964,53.895582,63.935743,78.286479,103.0,4.644391,2.757697
9f2484249a2811edaf6b121d2657380a,9f2484249a2811edaf6b121d2657380a,B148,lionesque_ghoulish_cob,9f2484249a2811edaf6b121d2657380a,B148-P2C3,TGTCTTTCCTGCCAG,singlet,1665,338,20192,...,7.418181,5470.0,8.607217,39.451554,54.661792,64.643510,77.111517,338.0,5.826000,6.179159


In [12]:
b_cells_subset['AIFI_L1'].unique()

['B cell']
Categories (9, object): ['B cell', 'DC', 'Erythrocyte', 'ILC', ..., 'NK cell', 'Platelet', 'Progenitor cell', 'T cell']

In [15]:
b_cells_subset['AIFI_L3'].unique()

['CD95 memory B cell', 'Transitional B cell', 'Core memory B cell', 'Activated memory B cell', 'CD27- effector B cell', ..., 'Platelet', 'Memory CD4 Treg', 'DN T cell', 'CMP cell', 'CD14+ cDC2']
Length: 43
Categories (71, object): ['ASDC', 'Activated memory B cell', 'Adaptive NK cell', 'BaEoMaP cell', ..., 'Transitional B cell', 'Type 2 polarized memory B cell', 'cDC1', 'pDC']

In [21]:
x = b_cells_subset['AIFI_L3'].value_counts()
x.to_csv("test.csv")

In [5]:
combined.to.csv("/home/jupyter/Myeloid_cells/files/AIFI_fluYr1Yr2_raw.csv")

AttributeError: 'AnnData' object has no attribute 'to'

In [20]:
combined.obs.to_parquet("/home/jupyter/Myeloid_cells/files/AIFI_fluYr1Yr2_raw.parquet")

In [22]:
import pandas as pd
df = pd.read_parquet('/home/jupyter/Myeloid_cells/files/AIFI_fluYr1Yr2_raw.parquet')
df.to_csv('/home/jupyter/Myeloid_cells/files/AIFI_fluYr1Yr2_raw.csv')

,barcodes,batch_id,cell_name,cell_uuid,chip_id,hto_barcode,hto_category,n_genes,n_mito_umis,n_reads,...,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mito,log1p_total_counts_mito,pct_counts_mito
barcodes,,,,,,,,,,,,,,,,,,,,,
70ced6d648b611eaa65d9e7d578d66f2,70ced6d648b611eaa65d9e7d578d66f2,B001,rhinestone_mannerly_argali,70ced6d648b611eaa65d9e7d578d66f2,B001-P1C1,TGATGGCCTATTGGG,singlet,2965,354,54033,...,7.994970,15237.0,9.631548,38.826541,54.997703,65.898799,77.134607,354.0,5.872118,2.323292
70da1ed848b611eaa65d9e7d578d66f2,70da1ed848b611eaa65d9e7d578d66f2,B001,ghastful_sphinxian_zopilote,70da1ed848b611eaa65d9e7d578d66f2,B001-P1C1,TGATGGCCTATTGGG,singlet,2691,311,44212,...,7.898040,12929.0,9.467305,39.771057,55.077732,65.906103,77.059324,311.0,5.743003,2.405445
c67f8e0248bd11eab2ee7a6edf799b61,c67f8e0248bd11eab2ee7a6edf799b61,B001,feudal_respected_meadowhawk,c67f8e0248bd11eab2ee7a6edf799b61,B001-P1C1,TGATGGCCTATTGGG,singlet,2495,197,32319,...,7.822445,10139.0,9.224243,38.485058,50.991222,61.623434,74.080284,197.0,5.288267,1.942992
c6aba74448bd11eab2ee7a6edf799b61,c6aba74448bd11eab2ee7a6edf799b61,B001,atrophied_piercing_piglet,c6aba74448bd11eab2ee7a6edf799b61,B001-P1C1,TGATGGCCTATTGGG,singlet,3620,663,61476,...,8.194506,18496.0,9.825364,35.218426,52.341047,63.078503,73.507785,663.0,6.498282,3.584559
5cfb5a3048b111ea8fbed2ddb8e0a14a,5cfb5a3048b111ea8fbed2ddb8e0a14a,B001,absent_unsweet_chihuahua,5cfb5a3048b111ea8fbed2ddb8e0a14a,B001-P1C1,TGATGGCCTATTGGG,singlet,2974,466,38227,...,7.997999,13444.0,9.506363,35.792919,51.919072,62.682237,73.728057,466.0,6.146329,3.466230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9f27aa829a2811edaf6b121d2657380a,9f27aa829a2811edaf6b121d2657380a,B148,small_forcible_mara,9f27aa829a2811edaf6b121d2657380a,B148-P2C3,TGTCTTTCCTGCCAG,singlet,2820,277,52036,...,7.944847,14314.0,9.569063,36.390946,52.012016,63.462345,75.869778,277.0,5.627621,1.935168
7794cb349a1f11edbb4f623fa13b87ec,7794cb349a1f11edbb4f623fa13b87ec,B148,treelike_whitefaced_cusimanse,7794cb349a1f11edbb4f623fa13b87ec,B148-P2C3,TGTCTTTCCTGCCAG,singlet,1396,0,22511,...,7.242082,6025.0,8.703838,46.224066,62.074689,72.946058,84.763485,0.0,0.000000,0.000000
779a401e9a1f11edbb4f623fa13b87ec,779a401e9a1f11edbb4f623fa13b87ec,B148,grapy_fibered_boutu,779a401e9a1f11edbb4f623fa13b87ec,B148-P2C3,TGTCTTTCCTGCCAG,singlet,3197,336,49543,...,8.070281,13860.0,9.536835,32.236652,46.464646,57.525253,70.367965,336.0,5.820083,2.424242


In [7]:
combined.obs.index=combined.obs['barcodes']

In [8]:
fitlered_gene=pd.DataFrame()
for i in combined.obs['AIFI_L3'].unique():
    print(i)
    adata_subset=combined[combined.obs['AIFI_L3']==i]
    sc.pp.filter_genes(adata_subset, min_cells=round(np.shape(adata_subset.X)[0]*0.1))
    gene_list=pd.DataFrame(list(adata_subset.var.index))
    gene_list.columns=['gene']
    gene_list['AIFI_L3']=i
    fitlered_gene=pd.concat([fitlered_gene,gene_list])


cDC1


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


KLRF1- effector Vd1 gdT


/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


ISG+ CD16 monocyte


...

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


DN T cell


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


ASDC


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [9]:
fitlered_gene.to_csv('filtered_gene_list.csv')